In [1]:
import requests
main_url="https://www.zara.com/se/sv/barn-flicka-nyheter-l391.html?v1=1282252"
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.62 Safari/537.36'}


In [2]:
import csv

f = open("zara.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","composition","color","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("zara.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,composition,color,url,img_url,article_no,blob_url


In [3]:
def quality(url_qu):
    #https://github.com/garywu/google-compute-engine-selenium
    # install chromium, its driver, and selenium
    # !sudo apt install chromium-chromedriver
    # !sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin
    # !pip install selenium
    # set options to be headless, ..

    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')



    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(url_qu)
    driver.set_window_size(1024, 768)


    button=driver.find_elements_by_xpath('/html/body/div[2]/section/section/div[1]/div/div[2]/div[3]/ul/li[1]/a')

    driver.execute_script("arguments[0].click();", button[0])


    try:

        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "popup-composition"))
        )
    except:
        print('error')
    # finally:
    #     driver.quit()

    material=driver.find_elements_by_xpath('//*[@id="popup-composition"]/ul[1]/li[1]/p[2]')
    composition=material[0].text
    return composition

In [4]:
from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [5]:
import urllib.request
import csv

import time
import random

def scrapper(atr,gender):
    r=requests.get(main_url, headers=headers)
    html_doc=r.content.decode()
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_doc, 'html.parser')
    cs=soup.find("li", attrs={ "data-name" :atr})
    html_doc2=str(cs.ul)
    soup2 = BeautifulSoup(html_doc2, 'html.parser')
    css=soup2.find_all("li",attrs={"data-layout":"products-category-view"})
    cgs=[]
    ur=[]
    for i in css:
        try:
            if i.a['href'] not in ur:
                cgs.append(i['data-name'])
                ur.append(i.a['href'])
#         print(i['data-name'])                                
#         print(i.a['href'])
            
        except:
            continue
#         print('error :',i)
    cgs.pop(0)
    ur.pop(0)
    for u,category in zip(ur,cgs):
        print("sub category url is",u)
        p=requests.get(u,headers=headers)
        html_doc2=p.content.decode()
        soup2 = BeautifulSoup(html_doc2, 'html.parser')

        acd=soup2.find("ul",attrs={"class":"product-list _productList"})

        html_doc3=str(acd)
        soup3 = BeautifulSoup(html_doc3, 'html.parser')
        error=[]
        for k in soup3.find_all("li"):
            try:
                brand='zara'
                article_no=k['data-productid']
                name=k.a.img['alt']
                url=k.a['href']
                price=k.find("span")['data-price']
                u=requests.get(url,headers=headers)
                html_doc3=u.content.decode()
                soup3 = BeautifulSoup(html_doc3, 'html.parser')
                print(article_no,url,price,name)
                u=requests.get(url,headers=headers)
                html_doc3=u.content.decode()
                soup3 = BeautifulSoup(html_doc3, 'html.parser')
                ig=soup3.find("a",attrs={"class":"_seoImg main-image"})
                img_url=ig['href']
                
                img_url = "https:" + img_url
                img_url=img_url.replace(' ','+')
                ac='zara'+'/'+article_no+str('.jpg')

                opener = urllib.request.URLopener()
                opener.addheader('User-Agent', 'whatever')
                filename, headers2 = opener.retrieve(img_url, ac)


                blob_name=str('zara')+str('/')+str(article_no)
                path_to_file='zara/'+article_no+str('.jpg')
                blob_url=upload_to_bucket(blob_name, path_to_file)
                
                color=soup3.find("span",attrs={"class":"_colorName"}).text
                composition=quality(url_qu=url)
                for s in soup3.find_all("span",attrs={"class":"size-name"}):
                    size=s.text
                    with open("zara.csv", "a", encoding='utf-8') as f:
    
                        writer = csv.writer(f)
                        writer.writerow([name,brand,gender,category,size,price,composition,color,url,img_url,article_no,blob_url])
                print(color,composition,size,img_url)

            except:
                print(k)
                continue

    
# scrapper(atr="POJKE | 6 - 14 år",gender="boy")
# url_data=["POJKE | 6 - 14 år","FLICKA | 6 - 14 år"]   
    

In [6]:
url_data=["POJKE | 6 - 14 år","FLICKA | 6 - 14 år","BABY FLICKA | 3 månader - 5 år","BABY POJKE | 3 månader - 5 år","MINI | 0 - 12 månader"]   




In [ ]:
scrapper(atr=url_data[0],gender="boy")


sub category url is https://www.zara.com/se/sv/barn-pojke-ytterklader-l231.html
23422850 https://www.zara.com/se/sv/denimjacka-p06987755.html 299,00 SEK DENIMJACKA


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Rödorange 100% BOMULL 13-14 år (164 cm) https://static.zara.net/photos///2019/I/0/3/p/6987/755/670/8/w/560/6987755670_2_10_1.jpg?ts=1562665297495
<li id="category-description-top"><h1 class="category-main-header">POJKJACKOR</h1><div class="category-description">De trendigaste pojkjackorna: quiltade dunjackor, bomberjackor, jeansjackor, pilotjackor och jackor för pojke med tryck. Beställ enkelt med fri retur</div></li>
12817669 https://www.zara.com/se/sv/denimjacka-p06987754.html 299,00 SEK DENIMJACKA


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Sandfärgad 100% BOMULL 13-14 år (164 cm) https://static.zara.net/photos///2019/I/0/3/p/6987/754/711/8/w/560/6987754711_2_10_1.jpg?ts=1563207131012
12817237 https://www.zara.com/se/sv/denimjacka-p06987755.html 299,00 SEK DENIMJACKA


...

Svart BASTYG
100% VISKOS 13-14 år (164 cm) https://static.zara.net/photos///2019/I/0/3/p/3548/660/800/8/w/560/3548660800_2_10_1.jpg?ts=1561619955964
13678699 https://www.zara.com/se/sv/jacka-i-sporttyg-p09347763.html 249,00 SEK JACKA I SPORTTYG


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Svart 100% POLYESTER 13-14 år (164 cm) https://static.zara.net/photos///2019/I/0/3/p/9347/763/800/8/w/560/9347763800_2_10_1.jpg?ts=1562745347673
12758176 https://www.zara.com/se/sv/kombinerad-sportjacka-p05992764.html 249,00 SEK KOMBINERAD SPORTJACKA


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  """Entry point for launching an IPython kernel.


Ljusgrön 100% POLYESTER 13-14 år (164 cm) https://static.zara.net/photos///2019/I/0/3/p/5992/764/526/8/w/560/5992764526_2_10_1.jpg?ts=1561475660248


In [ ]:
scrapper(atr=url_data[1],gender="girl")


In [ ]:
scrapper(atr=url_data[2],gender="girl")


In [ ]:
scrapper(atr=url_data[3],gender="boy")


In [ ]:
scrapper(atr=url_data[4],gender="unisex")
